In [1]:
import math
import pandas as pd
from collections import defaultdict
from tqdm import tqdm 

In [2]:
df = pd.read_pickle("../Datasets/USA_Filtered.pkl")

In [3]:
df_map = pd.read_csv("../Datasets/states_lat_lon.tsv", delimiter="\t")

In [4]:
df_map2 = pd.read_csv("../Datasets/state_abbrevation.tsv", delimiter="\t").rename(
    columns={"State": "state", 'Postal Abbr.': "code"})[["state", "code"]]

In [5]:
df_map_joined = df_map.merge(df_map2, how="inner", on="state")

In [6]:
def get_distance(lat1, lon1, lat2, lon2):
    rlat1 = math.pi * lat1 / 180
    rlat2 = math.pi * lat2 / 180
    theta = lon1 - lon2
    rtheta = math.pi * theta / 180
    dist = math.sin(rlat1) * math.sin(rlat2) + math.cos(rlat1) * math.cos(rlat2) * math.cos(rtheta)
    dist = math.acos(dist)
    dist = dist * 180 / math.pi
    dist = dist * 60 * 1.1515
    return dist

In [7]:
def get_state(gps):
    lat, lon = gps[0], gps[1]
    dist = []
    
    for i in range(len(df_map_joined)):
        dist.append((get_distance(lat1=lat, lon1=lon, 
                                  lat2=df_map_joined.iloc[i]["latitude"], 
                                  lon2=df_map_joined.iloc[i]["longitude"]), 
                     df_map_joined.iloc[i]["code"]))
    
    state = sorted(dist, key=lambda item: item[0])[0][1]
    return state

In [8]:
get_state([34.140884, -117.999102])

'CA'

In [9]:
tqdm.pandas()

In [ ]:
df2 = df[["lat", "lon", "gps"]].drop_duplicates(subset=["lat", "lon"])

In [ ]:
df2["state"] = df2["gps"].progress_map(get_state)